In [38]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import json
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import re
import json
import os
import numpy as np

In [2]:
with open(file='seven_test.json', mode='r', encoding='utf-8') as reader:
    store_list = json.load(reader)
temptab = pd.DataFrame.from_dict(store_list)
temptab.head()

,name,address
0,中坡,台北市南港區中坡南路47號1樓
1,中研,台北市南港區研究院路二段128號1樓(學術活動中心)
2,中貿,台北市南港區經貿二路186號2樓
3,玉成,台北市南港區西新里南港路三段3號1樓
4,玉德,台北市南港區玉成街150號1樓


In [3]:
print('number of seven list: ' + str(temptab.shape))

number of seven list: (10, 2)


## Open/Create the file
The main function will crashed because Google will block the robot! Thus, you can continue to finish the adjacency matrix by reading the json file again.

In [7]:
len(temptab)

10

In [9]:
if os.path.exists('adjacency_matrix_test.json'):
    print('open file')
    with open(file='adjacency_matrix_test.json', mode='r', encoding='utf-8') as reader:
        adjacency_matrix = json.load(reader)
    if len(adjacency_matrix) != len(temptab):
        print('Error! Different size!')
else:
    print('create file')
    adjacency_matrix = {}
    
    # run over all column
    for store_a in store_list:
        adjacency_matrix[store_a['name']] = {}
        # run over all row
        for store_b in store_list:
            adjacency_matrix[store_a['name']][store_b['name']] = -1
    # write it
    with open(file='adjacency_matrix_test.json', mode='w', encoding='utf-8') as writer:
        json.dump(obj=adjacency_matrix, fp=writer, indent=4)
        

tempadj = pd.DataFrame.from_dict(adjacency_matrix)
tempadj.head()

create file


,中坡,中研,中貿,玉成,玉德,向揚,庄研,佳樂,忠陽,昆陽
中坡,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
中研,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
中貿,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
玉成,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
玉德,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


## Convert to number

In [10]:
def STR_to_NUM(data):
    # the origin data as follwoing
    # 路/台5線\\",[2276,\\"2.3
    # 公里
    line = data.split('\"')
    return float(line[-1])

## Get distance

In [11]:
def get_dist(soup):
    # find the html contents
    doc = soup.prettify()
    
    if doc.find('公里') != -1:
        # find the position where we need
        initial_pos = doc.find('公里')
        # capture the range text we need
        data = doc[initial_pos-20:initial_pos]
        # return the number
        return STR_to_NUM(data)

    elif doc.find('公尺') != -1:
        initial_pos = doc.find('公尺')
        data = doc[initial_pos-20:initial_pos]
        return STR_to_NUM(data) / 1000
    
    else:
        return -1

## Main

In [12]:
# for robot and negative distance
flag = False
# for url template
template = 'https://www.google.com.tw/maps/dir/{}/{}'
# construct the adjacency matrix
numofstore = len(store_list)
for i, store_a in enumerate(store_list):
    if flag:
        break
    for j, store_b in enumerate(store_list):
        # diagonal is zero
        if store_a['name'] == store_b['name']:
            adjacency_matrix[store_a['name']][store_b['name']] = 0.0
            continue
        
        # this entry has computed
        if adjacency_matrix[store_a['name']][store_b['name']] > 0.0:
            continue
        
        # visit the web
        url = template.format(store_a['address'], store_b['address'])
        response = requests.get(url)
        
        # request NOT OK, BREAK!
        if response.status_code != 200:
            flag = True
            print('I\'m a robot')
            break
        
        # if request OK
        # extract link to tree structure
        soup = BeautifulSoup(response.text, 'html.parser')
        # find the distance
        dist = get_dist(soup)
        # assert distance > 0
        if dist <= 0:
            flag = True
            print('{} <-> {}'.format(store_a['name'], store_b['name']))
            print(url)
            print('Error! Distance is negative!')
            break
            
        # symmetrize the matrix
        adjacency_matrix[store_a['name']][store_b['name']] = dist
        adjacency_matrix[store_b['name']][store_a['name']] = dist
        # print the process
        it = numofstore*i+j+1
        print('process: '+str(it/np.array(numofstore*numofstore)))
        print('{} <-> {}: {} km'.format(store_a['name'], store_b['name'], format(dist)))
        print('-'*50)

print('Done!')

process: 0.02
中坡 <-> 中研: 5.5 km
--------------------------------------------------
process: 0.03
中坡 <-> 中貿: 4.7 km
--------------------------------------------------
process: 0.04
中坡 <-> 玉成: 2.2 km
--------------------------------------------------
process: 0.05
中坡 <-> 玉德: 0.35 km
--------------------------------------------------
process: 0.06
中坡 <-> 向揚: 2.4 km
--------------------------------------------------
process: 0.07
中坡 <-> 庄研: 5.6 km
--------------------------------------------------
process: 0.08
中坡 <-> 佳樂: 3.4 km
--------------------------------------------------
process: 0.09
中坡 <-> 忠陽: 2.0 km
--------------------------------------------------
process: 0.1
中坡 <-> 昆陽: 1.7 km
--------------------------------------------------
process: 0.13
中研 <-> 中貿: 3.0 km
--------------------------------------------------
process: 0.14
中研 <-> 玉成: 4.6 km
--------------------------------------------------
process: 0.15
中研 <-> 玉德: 5.5 km
--------------------------------------------------
proc

In [13]:
adj_mat = pd.DataFrame.from_dict(adjacency_matrix)
adj_mat.head()

,中坡,中研,中貿,玉成,玉德,向揚,庄研,佳樂,忠陽,昆陽
中坡,0.00,5.5,4.7,2.2,0.35,2.4,5.6,3.4,2.00,1.7
中研,5.50,0.0,3.0,4.6,5.50,4.5,1.3,2.9,4.10,4.7
中貿,4.70,3.0,0.0,3.4,4.50,3.3,2.6,1.5,3.00,3.6
玉成,2.20,4.6,3.4,0.0,2.30,0.4,4.5,1.6,0.45,0.5
玉德,0.35,5.5,4.5,2.3,0.00,2.2,5.3,3.1,1.80,1.4


In [14]:
with open(file='adjacency_matrix_test.json', mode='w', encoding='utf-8') as writer:
    json.dump(obj=adjacency_matrix, fp=writer, indent=4)